In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm

Make sure the model is deterministic 

In [2]:
_ = torch.manual_seed(0)

In [3]:
transform = transforms.Compose([transforms.ToTensor() ,

transforms.Normalize((0.1307,),(0.3081,))

])

# mnist_trainset = datasets.MNIST(root='./',train=True , download=True , transform= transform)

# train_loader = torch.utils.data.DataLoader(mnist_trainset , batch_size = 10 , shuffle = True)

# mnist_testset = datasets.MNIST(root='../data' , train = False , download = True , transform = transform)

# test_loader = torch.utils.data.DataLoader(mnist_testset,batch_size = 10 , shuffle = True)

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


mnist_trainset = datasets.MNIST(root='./data', train=True, download=False, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=False, transform=transform)

train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=False)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [23]:
class bigboy(nn.Module):

    def __init__(self , hidden_size_1 = 1000 , hidden_size_2 =2000):

        super(bigboy,self).__init__()
        self.linear1 = nn.Linear(28*28 , hidden_size_1)
        self.linear2 = nn.Linear(hidden_size_1 , hidden_size_2)
        self.linear3 = nn.Linear(hidden_size_2,10)
        self.relu = nn.ReLU()

    def forward(self , img):

        x = img.view(-1 , 28*28) # what does this do??

        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)

        return x

net = bigboy().to(device)

In [44]:
def train(train_loader , net , epochs=5 , total_iterations_limit = None):

    criteria = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(),lr=0.001)

    total_iterations = 0

    for epoch in range(epochs):
        
        net.train()

        loss_sum = 0
        num_iterations = 0
        data_iterator = tqdm(train_loader,desc=f'Epoch {epoch+1}')

        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:

            num_iterations+=1
            total_iterations+=1
            x,y = data
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            output = net(x.view(-1,28*28))

            loss = criteria(output,y)
            loss_sum += loss.item()

            avg_loss = loss_sum / num_iterations

            data_iterator.set_postfix(loss=avg_loss)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:

                return

train(train_loader,net,epochs=1)



Epoch 1: 100%|██████████| 595/595 [00:01<00:00, 427.03it/s, loss=0]


In [45]:
original_weights = {}

for name , param in net.named_parameters():

    original_weights[name] = param.clone().detach()

print(original_weights)

{'linear1.bias': tensor([-6.1826e-03, -1.1782e-02, -3.7209e-02,  3.5054e-02, -2.7781e-03,
        -4.1314e-02,  7.9050e-03,  3.1383e-02, -1.0069e-02, -4.8849e-03,
         3.2520e-02, -4.0118e-02, -1.1075e-02, -3.7919e-02,  1.5667e-02,
         1.5197e-02, -8.3135e-03, -4.3067e-03,  2.0399e-02, -2.5367e-02,
         2.6505e-02,  2.4834e-02,  4.6612e-03, -3.2186e-03,  1.2898e-02,
         3.5037e-02, -5.2149e-03,  3.5380e-03, -1.2504e-02, -2.7773e-02,
         4.1928e-03,  2.2047e-02, -3.1087e-02, -2.5898e-02, -1.6965e-02,
         2.8190e-02,  1.6634e-02, -2.3039e-02, -7.7869e-03,  4.3765e-03,
        -1.1344e-03, -4.4021e-03, -3.2640e-03,  1.1378e-02,  3.4757e-03,
        -3.3046e-02, -3.0458e-02, -3.1046e-02, -2.7696e-02,  2.7731e-02,
         1.5464e-02, -1.4977e-02,  2.2554e-03, -1.4871e-02,  2.2429e-02,
         3.2983e-02, -2.1136e-02, -9.8758e-03, -2.7631e-03,  2.3001e-02,
        -1.8290e-02, -2.8561e-02, -1.8739e-03, -1.5080e-02, -1.7763e-02,
         1.5463e-02, -2.2214e-02, 

In [46]:
def test():

    correct = 0
    total = 0

    wrong_counts = [0 for i in range(10)]

    with torch.no_grad():

        for data in tqdm(test_loader , desc='Testing'):

                x,y = data

                x = x.to(device)
                y = y.to(device)
                output = net(x.view(-1,784))

                for idx , i in enumerate(output):

                    if torch.argmax(i) == y[idx]:
                        correct += 1
                    else:
                        wrong_counts[y[idx]] +=1
                    total+=1
    print(f'Accuracy: {round(correct/total, 3)}')
    for i in range(len(wrong_counts)):
        print(f'wrong counts for the digit {i}: {wrong_counts[i]}')

test()

    

Testing: 100%|██████████| 1000/1000 [00:01<00:00, 564.83it/s]

Accuracy: 0.101
wrong counts for the digit 0: 980
wrong counts for the digit 1: 1135
wrong counts for the digit 2: 1032
wrong counts for the digit 3: 1010
wrong counts for the digit 4: 982
wrong counts for the digit 5: 892
wrong counts for the digit 6: 958
wrong counts for the digit 7: 1028
wrong counts for the digit 8: 974
wrong counts for the digit 9: 0


# Now lets calculate the total number of parameters

In [27]:
total_parameters_original = 0

for index , layer in enumerate([net.linear1 , net.linear2 , net.linear3]):

    total_parameters_original += layer.weight.nelement() + layer.bias.nelement()
    print(f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape}')

print(f'Total number of parameters: {total_parameters_original:,}')

Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000])
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000])
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10])
Total number of parameters: 2,807,010


lets define LoRA parameterization : https://pytorch.org/tutorials/intermediate/parametrizations.html

In [28]:
class LoRAParametrization(nn.Module):
    def __init__(self, features_in, features_out, rank=1, alpha=1, device='cpu'):
        super().__init__()
        # Section 4.1 of the paper: 
        #   We use a random Gaussian initialization for A and zero for B, so ∆W = BA is zero at the beginning of training
        self.lora_A = nn.Parameter(torch.zeros((rank,features_out)).to(device))
        self.lora_B = nn.Parameter(torch.zeros((features_in, rank)).to(device))
        nn.init.normal_(self.lora_A, mean=0, std=1)
        
        # Section 4.1 of the paper: 
        #   We then scale ∆Wx by α/r , where α is a constant in r. 
        #   When optimizing with Adam, tuning α is roughly the same as tuning the learning rate if we scale the initialization appropriately. 
        #   As a result, we simply set α to the first r we try and do not tune it. 
        #   This scaling helps to reduce the need to retune hyperparameters when we vary r.
        self.scale = alpha / rank
        self.enabled = True

    def forward(self, original_weights):
        if self.enabled:
            # Return W + (B*A)*scale
            return original_weights + torch.matmul(self.lora_B, self.lora_A).view(original_weights.shape) * self.scale
        else:
            return original_weights

In [42]:
import torch.nn.utils.parametrize as parametrize

def linear_layer_parameterization(layer, device, rank=1, lora_alpha=1):
    # Only add the parameterization to the weight matrix, ignore the Bias

    # From section 4.2 of the paper:
    #   We limit our study to only adapting the attention weights for downstream tasks and freeze the MLP modules (so they are not trained in downstream tasks) both for simplicity and parameter-efficiency.
    #   [...]
    #   We leave the empirical investigation of [...], and biases to a future work.
    
    features_in, features_out = layer.weight.shape
    return LoRAParametrization(
        features_in, features_out, rank=rank, alpha=lora_alpha, device=device
    )

parametrize.register_parametrization(
    net.linear1, "weight", linear_layer_parameterization(net.linear1, device)
)
parametrize.register_parametrization(
    net.linear2, "weight", linear_layer_parameterization(net.linear2, device)
)
parametrize.register_parametrization(
    net.linear3, "weight", linear_layer_parameterization(net.linear3, device)
)


def enable_disable_lora(enabled=True):
    for layer in [net.linear1, net.linear2, net.linear3]:
        layer.parametrizations["weight"][0].enabled = enabled

In [41]:
total_parameters_lora = 0
total_parameters_non_lora = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    total_parameters_lora += layer.parametrizations["weight"][0].lora_A.nelement() + layer.parametrizations["weight"][0].lora_B.nelement()
    total_parameters_non_lora += layer.weight.nelement() + layer.bias.nelement()
    print(
        f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape} + Lora_A: {layer.parametrizations["weight"][0].lora_A.shape} + Lora_B: {layer.parametrizations["weight"][0].lora_B.shape}'
    )
# The non-LoRA parameters count must match the original network
assert total_parameters_non_lora == total_parameters_original
print(f'Total number of parameters (original): {total_parameters_non_lora:,}')
print(f'Total number of parameters (original + LoRA): {total_parameters_lora + total_parameters_non_lora:,}')
print(f'Parameters introduced by LoRA: {total_parameters_lora:,}')
parameters_incremment = (total_parameters_lora / total_parameters_non_lora) * 100
print(f'Parameters incremment: {parameters_incremment:.3f}%')

Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000]) + Lora_A: torch.Size([1, 784]) + Lora_B: torch.Size([1000, 1])
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000]) + Lora_A: torch.Size([1, 1000]) + Lora_B: torch.Size([2000, 1])
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10]) + Lora_A: torch.Size([1, 2000]) + Lora_B: torch.Size([10, 1])
Total number of parameters (original): 2,807,010
Total number of parameters (original + LoRA): 2,813,804
Parameters introduced by LoRA: 6,794
Parameters incremment: 0.242%


In [40]:
for name, param in net.named_parameters():
    if 'lora' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

# Load the MNIST dataset again, by keeping only the digit 9
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
exclude_indices = mnist_trainset.targets == 9
mnist_trainset.data = mnist_trainset.data[exclude_indices]
mnist_trainset.targets = mnist_trainset.targets[exclude_indices]
# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# Train the network with LoRA only on the digit 9 and only for 100 batches (hoping that it would improve the performance on the digit 9)
train(train_loader, net, epochs=1, total_iterations_limit=100)

Freezing non-LoRA parameter linear1.bias
Freezing non-LoRA parameter linear1.parametrizations.weight.original
Freezing non-LoRA parameter linear2.bias
Freezing non-LoRA parameter linear2.parametrizations.weight.original
Freezing non-LoRA parameter linear3.bias
Freezing non-LoRA parameter linear3.parametrizations.weight.original


Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s, loss=0]

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [32]:
assert torch.all(net.linear1.parametrizations.weight.original == original_weights['linear1.weight'])
assert torch.all(net.linear2.parametrizations.weight.original == original_weights['linear2.weight'])
assert torch.all(net.linear3.parametrizations.weight.original == original_weights['linear3.weight'])

enable_disable_lora(enabled=True)
# The new linear1.weight is obtained by the "forward" function of our LoRA parametrization
# The original weights have been moved to net.linear1.parametrizations.weight.original
# More info here: https://pytorch.org/tutorials/intermediate/parametrizations.html#inspecting-a-parametrized-module
assert torch.equal(net.linear1.weight, net.linear1.parametrizations.weight.original + (net.linear1.parametrizations.weight[0].lora_B @ net.linear1.parametrizations.weight[0].lora_A) * net.linear1.parametrizations.weight[0].scale)

enable_disable_lora(enabled=False)
# If we disable LoRA, the linear1.weight is the original one
assert torch.equal(net.linear1.weight, original_weights['linear1.weight'])

In [33]:
enable_disable_lora(enabled=True)
test()

Testing: 100%|██████████| 1000/1000 [00:01<00:00, 567.41it/s]

Accuracy: 0.101
wrong counts for the digit 0: 980
wrong counts for the digit 1: 1135
wrong counts for the digit 2: 1032
wrong counts for the digit 3: 1010
wrong counts for the digit 4: 982
wrong counts for the digit 5: 892
wrong counts for the digit 6: 958
wrong counts for the digit 7: 1028
wrong counts for the digit 8: 974
wrong counts for the digit 9: 0


In [34]:
enable_disable_lora(enabled=False)
test()

Testing: 100%|██████████| 1000/1000 [00:01<00:00, 610.05it/s]

Accuracy: 0.101
wrong counts for the digit 0: 980
wrong counts for the digit 1: 1135
wrong counts for the digit 2: 1032
wrong counts for the digit 3: 1010
wrong counts for the digit 4: 982
wrong counts for the digit 5: 892
wrong counts for the digit 6: 958
wrong counts for the digit 7: 1028
wrong counts for the digit 8: 974
wrong counts for the digit 9: 0


Finetuning on 9 and 3

In [47]:
# for name, param in net.named_parameters():
#     if 'lora' not in name:
#         print(f'Freezing non-LoRA parameter {name}')
#         param.requires_grad = False

# # Load the MNIST dataset again, by keeping only the digit 9
# mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# exclude_indices = mnist_trainset.targets == 9
# mnist_trainset.data = mnist_trainset.data[exclude_indices]
# mnist_trainset.targets = mnist_trainset.targets[exclude_indices]
# # Create a dataloader for the training
# # include_indices = (mnist_trainset.targets == 9) | (mnist_trainset.targets == 3)

# # Apply the mask to both data and targets
# # mnist_trainset.data = mnist_trainset.data[include_indices]
# # mnist_trainset.targets = mnist_trainset.targets[include_indices]
# train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# # Train the network with LoRA only on the digit 9 and only for 100 batches (hoping that it would improve the performance on the digit 9)
# train(train_loader, net, epochs=1, total_iterations_limit=100)


for name, param in net.named_parameters():
    if 'lora' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

# Load the MNIST dataset again, by keeping only the digit 9
mnist_trainset = datasets.MNIST(root='./data', train=True, download=False, transform=transform)
exclude_indices = mnist_trainset.targets == 9
mnist_trainset.data = mnist_trainset.data[exclude_indices]
mnist_trainset.targets = mnist_trainset.targets[exclude_indices]
# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# Train the network with LoRA only on the digit 9 and only for 100 batches (hoping that it would improve the performance on the digit 9)
train(train_loader, net, epochs=1, total_iterations_limit=100)

Freezing non-LoRA parameter linear1.bias
Freezing non-LoRA parameter linear1.parametrizations.weight.original
Freezing non-LoRA parameter linear2.bias
Freezing non-LoRA parameter linear2.parametrizations.weight.original
Freezing non-LoRA parameter linear3.bias
Freezing non-LoRA parameter linear3.parametrizations.weight.original


Epoch 1:   0%|          | 0/595 [00:00<?, ?it/s]

Epoch 1:  99%|█████████▉| 99/100 [00:00<00:00, 412.13it/s, loss=0]


In [ ]:
enable_disable_lora(enabled=True)
test()

Testing: 100%|██████████| 1000/1000 [00:01<00:00, 626.88it/s]

Accuracy: 0.915
wrong counts for the digit 0: 57
wrong counts for the digit 1: 17
wrong counts for the digit 2: 126
wrong counts for the digit 3: 22
wrong counts for the digit 4: 164
wrong counts for the digit 5: 79
wrong counts for the digit 6: 33
wrong counts for the digit 7: 169
wrong counts for the digit 8: 157
wrong counts for the digit 9: 21


Now lets try lora on VGG architecture

In [46]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = nn.AdaptiveAvgPool2d((1, 1))(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [47]:
num_classes = 10
num_epochs = 10
batch_size = 16
learning_rate = 0.005

model = VGG16(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  


# Train the model
total_step = len(train_loader)

In [48]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 

RuntimeError: Given input size: (512x1x1). Calculated output size: (512x0x0). Output size is too small